In [1]:
%load_ext autoreload
%autoreload 2

In [11]:
import sys
sys.path.append('../')

import shutil
from train import run
import os
import itertools
import glob

def write_errormsg2file(msg, error_file_name):
    if not os.path.isfile(error_file_name):
        with open(error_file_name, 'w') as f:
            f.write(f'error : {msg}\n')
    else:
        with open(error_file_name, 'a') as f:
            f.write(f'error : {msg}\n')
            
def do_exps(exps_dict= None, general_opts= None, device= None, exp_dir = '../figs/test', save_special= False):    
    exp_idx = 0
    keys= list(exps.keys())
    
    val_list_list= []
    key_list = [] #eg: 'MODEL.MODEL_A.rotation_lambda'
    key_suffix_list= [] # eg: 'rotation_lambda'
    
    for key, val_list in exps_dict.items():
        key_list.append(key)
        key_suffix_list.append(key.split('.')[-1])
        
        val_list_list.append(val_list)
        
    attr_combination_list = [list(s) for s in itertools.product(*val_list_list)]
    
    print(f'number of total experiments : {len(attr_combination_list)}')
    
    count_okay=0
    count_not_okay=0
    for attr_combination in attr_combination_list:
        save_dir = f'{exp_dir}/'
        opts= []
        for idx in range(len(attr_combination)):
            opts += [key_list[idx], attr_combination[idx]]
            attr= attr_combination[idx]
            
            #####
            attr_is_list= False
            try:attr_is_list = isinstance(eval(attr), list)
            except:pass
            if attr_is_list:
                attr= '_'.join(list(map(str, eval(attr)))) ## [, ] should not be in the directory name because glob is sensitive to that !
            #####
            
            save_dir+= f'{key_suffix_list[idx]}({attr})@'
    
        save_dir = save_dir[:-1] # remove last '@' 

        exp_idx+=1
        opts_other= ['NAME', f'exp_idx({exp_idx})', 
                     'GENERAL.device', device, 
                     'GENERAL.save_dir', save_dir
                    ]
                     
        opts_other+= general_opts

        opts = opts_other + opts

        
        if len(glob.glob(f'{save_dir}/1_*.jpg'))!=0:
            count_okay+=1
            continue
        
        count_not_okay+=1
                    
        try:shutil.rmtree(save_dir)
        except:pass

        save_folder_name= save_dir.split('/')[-1]
        if len(save_folder_name)>255:
            print(f'\nFolder length is too long: len(results_saving_folder) -> {len(save_folder_name)} (<= 255)')
            print(save_folder_name)
        
        try:
            run(opts= opts, save_special=save_special)
        except:
            error_file_name = f'{exp_dir}/errors.txt'
            write_errormsg2file(f'ERROR : {save_dir}', error_file_name)
            print(f'ERROR : {save_dir}')
    print(count_okay)
    print(count_not_okay)

In [12]:
exp_dir= '../figs/mnistv16'
!rm -rf $exp_dir
!mkdir $exp_dir

In [ ]:
device = 'cuda:0'

exps = {
    'MODEL.MODEL_DECODER.upsample_net': ['bicubic_interp', 'learnable_transpose_conv', 'custom_v2'], #['bicubic_interp', 'learnable_transpose_conv', 'custom_v1'],    
    'MODEL.MODEL_H.lr_H': ['0.0', '1.0']
    }


general_opts= ['TRAIN.show_results_epoch', '5',
               'TRAIN.epochs', '150',
               
               'MODEL.MODEL_A.lambda_scale_factor', '4',
               'MODEL.MODEL_A.rotation_lambda', '10000.0',
               'MODEL.MODEL_H.T', '8',
               'DATASET.img_size', '128',
               'MODEL.MODEL_H.H_init', 'randn_FourierBased',
               'MODEL.MODEL_DECODER.name', 'genv1',
               'MODEL.MODEL_DECODER.channel_list', '[24, 12, 8, 4, 2]',
               'DATASET.name', 'mnistdigits_grid2patch',
              ]

do_exps(exps, general_opts, device, exp_dir = exp_dir, save_special= False)

yt range (train): [2.607506758067757e-05 33.58327102661133]
yt range (test): [2.607506758067757e-05 33.68766403198242]
training loop time (for epoch: 4): 4.24759578704834 sec
validation loop time (for epoch: 4): 0.35028958320617676 sec


m : 1
